<h1> UCI - I&C SCI_X426.53 (Fall2017) </h1>
**Course Instructors:** <br>
Ted Pham (email:tedp@uci.edu) <br>
William Henry  (will@henryanalytics.com)



<h2>Week7 Assignment</h2>


---
This is an individual assignment, but feel free to exchange ideas on the forum.
__Name:__  Your Name Here<br>
__Email:__  peter_anteater@uci.edu   
__Week:__   7

__Due Time:__ Friday Dec 21 10am PST

# Overview

In this notebook, we will go over examples of running Spark and do some exercises
    1. KDnuggets Tutorial
    2. WordCount Exercise
    3. K-means Example (Optional, will be posted Monday Dec 11)
This notebook was tested on AWS EC2 jupyter interface using UCI AMI week7.




# 0. Initialize Spark

In [1]:
#Import SparkContext from pyspark
import re
from pyspark import SparkContext
sc = SparkContext()

In [2]:
# Try Week6 code
from operator import add
rdd1 = sc.parallelize([("a",1),("b",1),("a",1)])
sorted(rdd1.reduceByKey(add).collect())

[('a', 2), ('b', 1)]

# 1. KDnuggets Spark Tutorial

kdnuggets.com has an excellent tutorial on Spark and Python
https://www.kdnuggets.com/2015/11/introduction-spark-python.html

Due to copyrights, please read the tutorial and we follow along a slightly modified code here.

Starting with downloading the daily_show_guests.csv. 

In [10]:
!curl -L https://raw.githubusercontent.com/fivethirtyeight/data/master/daily-show-guests/daily_show_guests.csv -o daily.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  123k  100  123k    0     0   123k      0  0:00:01 --:--:--  0:00:01 1767k


In [11]:
!head -10 daily.csv

YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List
1999,actor,1/11/99,Acting,Michael J. Fox
1999,Comedian,1/12/99,Comedy,Sandra Bernhard
1999,television actress,1/13/99,Acting,Tracey Ullman
1999,film actress,1/14/99,Acting,Gillian Anderson
1999,actor,1/18/99,Acting,David Alan Grier
1999,actor,1/19/99,Acting,William Baldwin
1999,Singer-lyricist,1/20/99,Musician,Michael Stipe
1999,model,1/21/99,Media,Carmen Electra
1999,actor,1/25/99,Acting,Matthew Lillard


In [12]:
#-get the first 5 elements from the raw RDD
raw = sc.textFile("daily.csv")
raw.take(5)

['YEAR,GoogleKnowlege_Occupation,Show,Group,Raw_Guest_List',
 '1999,actor,1/11/99,Acting,Michael J. Fox',
 '1999,Comedian,1/12/99,Comedy,Sandra Bernhard',
 '1999,television actress,1/13/99,Acting,Tracey Ullman',
 '1999,film actress,1/14/99,Acting,Gillian Anderson']

In [13]:
#-split elements by commas
daily = raw.map(lambda line: line.split(','))
daily.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '1/11/99', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '1/12/99', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '1/13/99', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '1/14/99', 'Acting', 'Gillian Anderson']]

In [14]:
# Aggregate total count of visitors per year
tally = daily.map(lambda x: (x[0], 1))\
             .reduceByKey(lambda x,y: x+y)
print(tally)

PythonRDD[23] at RDD at PythonRDD.scala:48


In [15]:
# because Spark is lazy we need to perform an action on the RDD
tally.take(tally.count())

[('YEAR', 1),
 ('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163),
 ('2015', 100),
 ('1999', 166),
 ('2000', 169),
 ('2001', 157),
 ('2005', 162),
 ('2006', 161),
 ('2008', 164),
 ('2009', 163)]

**Question 1 (5pts)**
Sort the tally by year

In [16]:
#Student Code
tally.sortByKey(0)

PythonRDD[33] at RDD at PythonRDD.scala:48

In [18]:
tally.take(10)

[('YEAR', 1),
 ('2002', 159),
 ('2003', 166),
 ('2004', 164),
 ('2007', 141),
 ('2010', 165),
 ('2011', 163),
 ('2012', 164),
 ('2013', 166),
 ('2014', 163)]

In [19]:
#Filter out empty profession, make profession lowercase
# and get profession count for the visitors
filtered_daily = daily.filter(lambda line: line[1] != '') \
                      .map(lambda line: (line[1].lower(), 1))\
                      .reduceByKey(lambda x,y: x+y)
        
# get first 5 profession counts
filtered_daily.take(5)

[('actor', 596),
 ('film actress', 21),
 ('model', 9),
 ('stand-up comedian', 44),
 ('actress', 271)]

**Question 2 (5pts)** Get all professions and counts



In [23]:
#-Student Code-#
print (filtered_daily.collect())

[('actor', 596), ('film actress', 21), ('model', 9), ('stand-up comedian', 44), ('actress', 271), ('television personality', 13), ('comic', 2), ('musician', 19), ('film actor', 19), ('journalist', 253), ('us senator', 50), ('vocalist', 2), ('film director', 24), ('singer', 23), ('muppet', 2), ('american television personality', 3), ('rapper', 10), ('football player', 1), ('former mayor of cincinatti', 2), ('businesswoman', 2), ('american politician', 8), ('radio personality', 3), ('commentator', 24), ('reality show contestant', 1), ('magician', 1), ('former u.s. representative', 7), ('former united states secretary of labor', 6), ('band', 2), ('american football quarterback', 3), ('diplomat', 10), ('white house sommunications director', 5), ('sportscaster', 4), ('television producer', 3), ('screenwriter', 6), ('essayist', 2), ('minister', 4), ('host', 1), ('assistant secretary of defense', 1), ('former assistant to the president for communications', 3), ('basketball player', 12), ('edi

# 2. WordCount Exercise


In [24]:
# We will perform word count using Spark with Christmas Carol Text
# If you are working with the notebook on your local computer instead 
# of EC2 you can download the text from https://www.gutenberg.org/files/46/46-h/46-h.htm
# We look at first 10 lines
!head -10 christmas.txt

A CHRISTMAS CAROL

IN PROSE
BEING
A Ghost Story of Christmas

by Charles Dickens





**Perform some actions on Christmas Carol RDD***

In [25]:
# Create a textfile RDD
christmas = sc.textFile("christmas.txt")

#simple actions on the RDD
print(christmas.count())
print(christmas.first())
print(christmas.take(10))

4208
A CHRISTMAS CAROL
['A CHRISTMAS CAROL', '', 'IN PROSE', 'BEING', 'A Ghost Story of Christmas', '', 'by Charles Dickens', '', '', '']


**Question 3 (10 pts)**
Use Spark, get the 20 most common "lowercased" words (don't count stopwords)

Hint: Suggested Pseudocode, you're welcome to do your own

1. Define a function "findWord" that takes a line as input and return the words & their counts (if stopwords, don't count the word). Use assignment 4 solution the regex code and stopwords.

2. define a count RDD as:
    a. flatmap(find,Word)
    b. aggregate by key, add count
    c. switch key value pair to value,key pair
    d. use transformation sortByKey
3. collect the first 20 elements of the count RDD
4. collect the last 20 elements of the count RDD

In [43]:
def findWord(line):
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    line = pattern.sub('',line)
    line = line.split(' ')
    word_count = {}
    for word in line:
        if word in word_count:
            word_count[word]+=1
        else:
            word_count[word]=1
    return word_count
    

In [51]:
find_word=christmas.sflatMap(lambda x: findWord(x))

In [61]:
christmas_split=christmas.map(lambda line: line.split(',')).flatMap(lambda line: findWord(line))

In [77]:
christmas_wrd_cnt=christmas.flatMap(lambda line: line.split()).map(lambda w: (w,1)).reduceByKey(lambda v1, v2 :v1+v2).sortByKey(1).collect()

In [78]:
christmas_wrd_cnt[:20]

[('"\'And', 1),
 ('"A', 14),
 ('"Ah!"', 1),
 ('"All', 1),
 ('"Always', 1),
 ('"Am', 2),
 ('"An', 1),
 ('"And', 14),
 ('"Are', 6),
 ('"As', 1),
 ('"Ask', 1),
 ('"At', 3),
 ('"Bad,"', 1),
 ('"Bah!"', 2),
 ('"Bear', 1),
 ('"Because', 3),
 ('"Because,"', 1),
 ('"Bed-curtains!"', 2),
 ('"Before', 1),
 ('"Belle,"', 1)]

In [79]:
christmas_wrd_cnt[-20:]

[('you.', 8),
 ('you."', 2),
 ('you:', 1),
 ('you;', 2),
 ('you?', 3),
 ('you?"', 7),
 ('young', 25),
 ('young)', 1),
 ('young.', 1),
 ('younger', 3),
 ('youngest', 1),
 ('your', 54),
 ('yours', 1),
 ('yours,"', 1),
 ('yours?"', 1),
 ('yourself', 2),
 ('yourself,', 1),
 ('yourself?', 1),
 ('youth', 1),
 ('zeal.', 1)]

**Question 4 (10 pts)**
Use Spark, get the 20 least common "lowercased" words (don't count stopwords)

In [84]:
christmas_wrd_cnt=christmas.flatMap(lambda line: line.split()).map(lambda w: (w,1)).reduceByKey(lambda v1, v2 :v1+v2).sortByKey(2).collect()

In [85]:
christmas_wrd_cnt[:20]

[('"\'And', 1),
 ('"A', 14),
 ('"Ah!"', 1),
 ('"All', 1),
 ('"Always', 1),
 ('"Am', 2),
 ('"An', 1),
 ('"And', 14),
 ('"Are', 6),
 ('"As', 1),
 ('"Ask', 1),
 ('"At', 3),
 ('"Bad,"', 1),
 ('"Bah!"', 2),
 ('"Bear', 1),
 ('"Because', 3),
 ('"Because,"', 1),
 ('"Bed-curtains!"', 2),
 ('"Before', 1),
 ('"Belle,"', 1)]

**Question 5 (5pts)**
Use Spark, only counts the words that start with a Uppercase letter and print out the top 10 of those words.

In [ ]:
christmas_wrd_cnt[-20:0]

In [ ]:
# stop sparkcontext
sc.stop()